# Social Media Analytics - Group Project 

###### Group 5: Salma BARKETALLAH, Ndjaidoukmi KEDA, Noémie GAUTIER

Company: Google

#### I. Access Token & Installing Packages

In [1]:
# Getting tokens from the doc created:
source("tokens.R")

In [2]:
options(warnings = -1)

In [4]:
# Install rtweet package:
if(!require("rtweet")) install.packages("rtweet", quiet=TRUE); library("rtweet", quiet=TRUE)

In [5]:
# create the access token (authentication)
twitter_token <- create_token(
  app = app_name,
  consumer_key = consumer_key,
  consumer_secret = consumer_secret,
  access_token = access_token,
  access_secret = access_secret,
  set_renv=FALSE)

In [8]:
# Install Other packages:
for (i in c('SnowballC','slam','tm','Matrix','tidytext','dplyr','hunspell','purrr','wordcloud','RWeka')){
  if (!require(i, character.only=TRUE)) install.packages(i, repos = "http://cran.us.r-project.org", quiet=TRUE)
  require(i, character.only=TRUE, quiet=TRUE)
}

In [9]:
library(tidyverse, quiet=TRUE)

In [11]:
if (!require("topicmodels")) install.packages("topicmodels", quiet=TRUE) ; require("topicmodels")

In [23]:
library(dplyr)

#### II.  Gather Data on Google Via Twitter:

In [158]:
account <- 'Google lang:en'
h_account <- '#HeyGoogle lang:en' #the official name with hashtag
at_account <- '@google'
list_ID_number <- 20566259 #Google Around the World lis ID
list_ID_number2 <- 42906849 #Google Products list ID

###### Number of followers:

In [159]:
#When using the direct line of code to get more than 75000 followers we are having sleep time error so we have used it in the for loop to avoid this error.
# As google have more than 22millions of followers for our analysis we have limited the number to 150.000 users to be able to process
followersList <- list()
i <- 0
for (i in 1:10){
  if (i==1){
    followersList[[1]] <- get_followers("Google", n= 75000, page = "-1", retryonratelimit = TRUE, token=twitter_token)
  }
  else{
    nextpage <- next_cursor(followersList[[i-1]])
    Sys.sleep(15 * 60) ## Suspend execution of R expressions for 15 mins
    followersList[[i]] <- get_followers("Google", n= 75000, page = nextpage, retryonratelimit = TRUE, token=twitter_token)
  }

}

#combine all the list in one
followers <- rbindlist(followersList)
n_rows(followers)

user_id
<chr>
NA
NA
NA


n
<int>
6


In [0]:
#Collect profile data from those followers
followers_info <- list()
i <- 0
for (i in 1:10){
  followers_info[i] <- lapply(followersList[[i]],lookup_users,token=twitter_token)
}

followers_infodata <- rbindlist(followers_info)
nrow(followers_infodata)
str(followers_infodata)

In [0]:
#Save the data on csv  file
google_followers <- file.path("C:","Users", "nkeda" , "Desktop","google_followers"  )
write_as_csv(followers_google, google_followers, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

#save followers ID list
google_followersID <- file.path("C:","Users", "nkeda" , "Desktop","google_followersID"  )
write_as_csv(followers_google, google_followersID, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

###### User IDs of accounts followed by Google

In [160]:
# Google have less than 500 friends 
friends <- get_friends("Google", n= 500, page = "-1", retryonratelimit = TRUE, token=twitter_token)

head(friends, n=3)

<0 x 0 matrix>

In [0]:
#Collect friends profile data
friends_info <- lapply(friends,lookup_users,token=twitter_token)
friends_data <- rbindlist(friends_info)

In [0]:
#Save the data on CSV file
google_friends_data <- file.path("C:","Users", "nkeda" , "Desktop","google_friends_data"  )
write_as_csv(friends_data, google_friends_data, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

#Save Friends ID list

google_friendsID <- file.path("C:","Users", "nkeda" , "Desktop","google_friendsID"  )
write_as_csv(friends_data, google_friendsID, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

###### Gather all tweets from Google in 2020:

###### Tweets Mentioning Google:

In [0]:
# download tweets and hastags over the year (to be run once as we have a sandbox, only 50 requests are allowed)
# The idea here is to collect data over the year as we are limited with the number of time we can make the request with our sandbox,
# so we have decided in our code to loop over each month to collect data
tweetlist <-list()
htweetlist <- list()
years <-0
i <- 0

for (i in 1:12){
  years <- 202000 + i
  year_month <- as.character(years)
  start_date <- paste0(year_month,'010000')
  if (i==2){
    end_date <- paste0(year_month,'280000')
  } else {
    end_date <- paste0(year_month,'300000')
  }
  tweetlist[[i]] <- search_fullarchive(account, env_name="searchfull",fromDate=start_date,toDate=end_date,token=twitter_token)

  htweetlist[[i]] <-search_fullarchive(h_account, env_name="searchfull",fromDate=start_date,toDate=end_date,token=twitter_token)
}

tweets <- rbindlist(tweetlist)
htweets <- rbindlist(htweetlist)
#Save data as CSV file
google_tweets <- file.path("C:","Users", "nkeda" , "Desktop","google_tweets"  )
write_as_csv(tweets, google_tweets, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

#Save data as CSV file
google_htweets <- file.path("C:","Users", "nkeda" , "Desktop","google_htweets"  )
write_as_csv(htweets, google_htweets, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

In [0]:
#code to collecte tweets with @google over the year (to be run once as we have a sandbox, only 15 requests are allowed)
# same approach as previous one
atweetlist <-list()
years <-0
i <- 0

for (i in 1:12){
  years <- 202000 + i
  year_month <- as.character(years)
  start_date <- paste0(year_month,'010000')
  if (i==2){
    end_date <- paste0(year_month,'280000')
  } else {
    end_date <- paste0(year_month,'300000')
  }
  atweetlist[[i]] <- search_fullarchive(at_account, env_name="searchfull",fromDate=start_date,toDate=end_date,token=twitter_token)

}

#Save data as CSV file
google_atweets <- file.path("C:","Users", "nkeda" , "Desktop","google_atweets"  )
write_as_csv(atweets, google_atweets, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")

###### Timeline:

In [0]:
g_timeline <- get_timeline("Google", n=5000,token=twitter_token )

head(g_timeline)

In [0]:
#Save data on csv file

google_timeline <- file.path("C:","Users", "nkeda" , "Desktop","google_timeline"  )
write_as_csv(g_timeline, google_timeline, prepend_ids = TRUE, na = "", fileEncoding = "UTF-8")
